In [415]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [416]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [417]:
train=pd.read_csv("../input/spaceship-titanic/train.csv")
test=pd.read_csv("../input/spaceship-titanic/test.csv")
ss=pd.read_csv("../input/spaceship-titanic/sample_submission.csv")

In [418]:
print(train.shape)
train.head()

In [419]:
train.info()

In [420]:
ytrain=train.Transported
ytrain=ytrain.astype(int)
ytrain

In [421]:
print(test.shape)
passid=test.PassengerId
test.head()

In [422]:
test.info()

In [423]:
train.hist(bins=50, figsize=(20,15))

In [424]:
train.describe()

In [425]:
train.drop("Transported",axis=1,inplace=True)
dataset=pd.concat([train,test],axis=0)
dataset

In [426]:
sns.heatmap(dataset.isnull())

In [427]:
dataset["Cabin"].value_counts
dataset.Cabin.nunique()

In [428]:
dataset=dataset.drop(['Name','Cabin'],axis=1)

In [429]:
col=['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Age']
dataset[col]=dataset[col].fillna(dataset[col].median())
dataset.isnull().sum()

In [430]:
dataset["HomePlanet"].value_counts()

In [431]:
dataset["CryoSleep"].value_counts()

In [432]:
dataset["Destination"].value_counts()

In [433]:
dataset["VIP"].value_counts()

In [434]:
fix,ax = plt.subplots(2, 2, figsize = (20, 20))

sns.countplot(ax=ax[0,0],x="HomePlanet",data=dataset.fillna("na"))
sns.countplot(ax=ax[0,1],x="VIP",data=dataset.fillna("na"))
sns.countplot(ax=ax[1,0],x="CryoSleep",data=dataset.fillna("na"))
sns.countplot(ax=ax[1,1],x="Destination",data=dataset.fillna("na"))

In [435]:
dataset=dataset.fillna({'HomePlanet':'Earth','CryoSleep':'False','Destination':'TRAPPIST-1e','VIP':'False'})
sns.heatmap(dataset.isnull())

In [436]:
dataset= dataset.astype({"CryoSleep":'bool', "VIP":'bool'})
bool1= {True:1,False:0}
planet= {"Europa":1,'Earth':2,'Mars':3}
dest= {"TRAPPIST-1e":1,"PSO J318.5-22":2,"55 Cancri e":3}

In [437]:
dataset['HomePlanet'] = dataset['HomePlanet'].map(planet)
dataset['CryoSleep'] = dataset['CryoSleep'].map(bool1)
dataset['Destination'] =dataset['Destination'].map(dest)
dataset['VIP'] =dataset['VIP'].map(bool1)
sns.heatmap(dataset.corr(),annot=True)
train.info()

In [438]:
dataset.drop('PassengerId',axis=1,inplace=True)
train_set=dataset.iloc[:8693,]
train_set.info()
test_set=dataset.iloc[8693:,]
test_set

In [439]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(train_set,ytrain,train_size=0.8,random_state=1)

In [440]:
from sklearn.preprocessing import StandardScaler
x_train=StandardScaler().fit_transform(x_train)
x_test=StandardScaler().fit_transform(x_test)
test_set=StandardScaler().fit_transform(test_set)

In [441]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model=LogisticRegression(max_iter=10000)
model.fit(x_train, y_train)
y_pred=model.predict(x_test)
acc=accuracy_score(y_test,y_pred)
print("accuracy score of logistic regression:",acc)

In [442]:
from sklearn.ensemble import RandomForestClassifier
rc=RandomForestClassifier(max_depth=5,random_state=10)
rc.fit(x_train,y_train)
ypred=rc.predict(x_test)
ac=accuracy_score(ypred,y_test)
print("accuracy score of randomforestclassifier:",ac)

In [443]:
from sklearn.tree import DecisionTreeClassifier
dc=DecisionTreeClassifier(criterion="entropy",random_state=5)
dc.fit(x_train,y_train)
p=dc.predict(x_test)
a=accuracy_score(p,y_test)
print("accuracy score of decisiontreeclassifier:",a)

In [444]:
from sklearn import svm
sv=svm.SVC(kernel='rbf')
sv.fit(x_train,y_train)
predict=sv.predict(x_test)
ac_=accuracy_score(predict,y_test)
print("accuracy score of svm:",ac_)

In [445]:
plt.bar(np.array(["logistic regression","SVM","random forest","decision tree"]),np.array([acc,ac_,ac,a]))
plt.show()

**we use logistic regression model as it has better accuracy score**

In [446]:
ss.head()

In [447]:
transportedpred=model.predict(test_set)
transportedpredframe=pd.DataFrame(transportedpred, columns=['Transported'])
transportedpredframe['Transported']=transportedpredframe['Transported'].replace(1, "True")
transportedpredframe['Transported']=transportedpredframe['Transported'].replace(0, "False")
transportedpredframe

In [448]:
pass_id=pd.DataFrame(passid, columns=['PassengerId'])
submission=pd.concat([pass_id,transportedpredframe],axis=1)

In [449]:
submission.to_csv("submission.csv", index=False)